In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [1]:
import keras


Using TensorFlow backend.


In [4]:
data = pd.read_csv('cleaned_data.csv')

In [5]:
data.head()

,Unnamed: 0,id,score,author,tweet,CleanedText
0,0,0,0,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,1,1,0,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,2,2,0,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,3,3,0,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass not behav mad see
4,4,4,0,joy_wolf,@Kwesidei not the whole crew,kwesidei not whole crew


In [6]:

# Keeping only the neccessary columns
data = data[['tweet','score']]

In [7]:
data.head()

,tweet,score
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0


In [8]:
data.tail()

,tweet,score
495901,My GrandMa is making Dinenr with my Mum,4
495902,Mid-morning snack time... A bowl of cheese noo...,4
495903,@ShaDeLa same here say it like from the Termi...,4
495904,@DestinyHope92 im great thaanks wbuu?,4
495905,cant wait til her date this weekend,4


In [13]:
data['tweet'] = data['tweet'].apply((lambda x: re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str(x), flags=re.MULTILINE)))
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['tweet'] = data['tweet'].apply((lambda x: re.sub('\s+',' ', x)))

In [14]:
data.head()

,tweet,score
0,is upset that he cant update his facebook by t...,0
1,kenichan i dived many times for the ball manag...,0
2,my whole body feels itchy and like its on fire,0
3,nationwideclass no its not behaving at all im ...,0
4,kwesidei not the whole crew,0


In [15]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

In [17]:
X.shape

(495906, 40)

In [18]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 40, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
Y = pd.get_dummies(data['score']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(332257, 40) (332257, 2)
(163649, 40) (163649, 2)


In [22]:
#print(Y_train[ Y_train[0] == 0].size)
#print(data[ data['sentiment'] == 'Negative'].size)

SyntaxError: unexpected EOF while parsing (<ipython-input-22-9c8e1bad806e>, line 2)

In [23]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 1028s - loss: 0.4799 - acc: 0.7680
Epoch 2/10
 - 1014s - loss: 0.4491 - acc: 0.7872
Epoch 3/10


KeyboardInterrupt: 

In [24]:
batch_size = 1024
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
 - 746s - loss: 0.4268 - acc: 0.8006
Epoch 2/10
 - 764s - loss: 0.4236 - acc: 0.8021
Epoch 3/10
 - 730s - loss: 0.4219 - acc: 0.8033
Epoch 4/10
 - 794s - loss: 0.4201 - acc: 0.8037
Epoch 5/10
 - 839s - loss: 0.4182 - acc: 0.8051
Epoch 6/10
 - 758s - loss: 0.4162 - acc: 0.8061
Epoch 7/10
 - 793s - loss: 0.4148 - acc: 0.8067
Epoch 8/10
 - 825s - loss: 0.4128 - acc: 0.8076
Epoch 9/10
 - 839s - loss: 0.4113 - acc: 0.8082
Epoch 10/10
 - 729s - loss: 0.4093 - acc: 0.8096


In [25]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.44
acc: 0.80


In [30]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=40, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 4):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  206 1142   12  199    8   75 1576   75   32   12  199]]
negative


In [2]:
import pickle
def savetofile(obj,filename):
    pickle.dump(obj,open(filename+".p","wb"))
def openfromfile(filename):
    temp = pickle.load(open(filename+".p","rb"))
    return temp

In [32]:
savetofile(tokenizer,"rnntokenizer")

In [33]:
savetofile(model,"rnnmodel")

In [3]:
tokenizer = openfromfile("rnntokenizer")
model = openfromfile("rnnmodel")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [41]:
twt = ['Yesterday I was sad but now I am happy']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=40, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0 278   1  26 114  19
   31   1  56  99]]
positive


In [10]:
twt = ['  massacre']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=40, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]
positive
